In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
# sys.path.append('../pubmed_rag')
# from utils import get_chunks
# from bioc import (
#     collapse_sections,
#     get_smaller_texts
# )


get the data for the vx db first. 

Note: The primary key and vector fields use their default names (“id” and “vector”).

to insert into vx database need each row as a dictionary

In [30]:
# read in
filename = '/Users/anglup/GitHub/pubmed-rag/biocjson/tester.csv'
df = pd.read_csv(filename, )
# rename id and vertex cols
df = df.rename(
    columns={
        'Unnamed: 0':'id',
        'embedding':'vertex'
    }, 
)
# ensure vertex is a list
df['vertex'] = df['vertex'].apply(literal_eval)

# sanity check
#data.head()

# to list of dicts
data = df.to_dict(orient='records')

to insert into vx database need each row as a dictionary

In [2]:
from pymilvus import MilvusClient

In [4]:
client = MilvusClient('demo.db')

if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [ ]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)
